# Homework 8
# LTCM Risk Decomposition

In [71]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import kurtosis, skew
from scipy.stats import norm
import seaborn as sns
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.precision",4)
sns.set(rc={'figure.figsize':(15,10)})

%matplotlib inline


import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[15, 6]
import matplotlib.cm as cm

In [72]:
# Getting gross and net returns
gmo_spy_file = 'C:/Users/dcste/OneDrive/Portfolio_Theory/Homework_Jupyter/portfolio_theory/gmo_analysis_data.xlsx'
file_ltcm = 'C:/Users/dcste/OneDrive/Portfolio_Theory/Homework_Jupyter/portfolio_theory/ltcm_exhibits_data.xlsx'
ltcm = pd.read_excel(file_ltcm,sheet_name='Exhibit 2', skiprows=2).rename(columns={'Unnamed: 0': "Date"})\
    .set_index("Date").dropna()
spy_rets = pd.read_excel(gmo_spy_file,sheet_name=2).rename(columns = {'Unnamed: 0': 'Date'}).set_index("Date")
rf = pd.read_excel(gmo_spy_file,sheet_name=3).rename(columns={'Unnamed: 0': 'Date'}).set_index("Date")


In [73]:
spy_rets['rf'] = rf['US3M']
spy_ex = spy_rets.subtract(spy_rets['rf'], axis = 0)['SPY']
spy_ex = pd.DataFrame(spy_ex).rename(columns = {'SPY':'SPY EXCESS'})


In [74]:
spy_ex.head()

,SPY EXCESS
Date,
1993-02-28,0.0082
1993-03-31,0.0199
1993-04-30,-0.0281
1993-05-31,0.0244
1993-06-30,0.0011


In [75]:
ltcm.head()

,Fund Capital ($billions),Gross Monthly Performancea,Net Monthly Performanceb,Index of Net Performance
Date,,,,
1994-03-01 00:00:00,1.1,-0.011,-0.013,0.99
1994-04-01 00:00:00,1.1,0.014,0.008,1.00
1994-05-01 00:00:00,1.2,0.068,0.053,1.05
1994-06-01 00:00:00,1.2,-0.039,-0.029,1.02
1994-07-01 00:00:00,1.4,0.116,0.084,1.10


In [76]:
ltcm.index = spy_ex.loc['1994-03':'1998-07'].index
ltcm_ex = ltcm.subtract(spy_rets['rf'], axis = 0).dropna().drop(columns = ['Index of Net Performance'])
ltcm_ex['SPY (excess)'] = spy_ex['SPY EXCESS']
ltcm_ex.columns = ['Fund Capital ($billions)','Gross Performance (Excess)', 'Net Performance (Excess)',"SPY (Excess)"]

In [77]:
ltcm_ex.head()

,Fund Capital ($billions),Gross Performance (Excess),Net Performance (Excess),SPY (Excess)
Date,,,,
1994-03-31,1.0970,-0.0140,-0.0160,-0.0449
1994-04-30,1.0967,0.0107,0.0047,0.0079
1994-05-31,1.1964,0.0644,0.0494,0.0123
1994-06-30,1.1965,-0.0425,-0.0326,-0.0264
1994-07-31,1.3963,0.1123,0.0803,0.0287


In [78]:
def summary_stats(df, annual_frac = 12):
    stat_df = (df.mean()*annual_frac).to_frame("Mean")
    stat_df['Vol'] = df.std()*np.sqrt(annual_frac)
    stat_df['Sharpe'] = stat_df['Mean']/stat_df['Vol']
    
    return stat_df

def tail_risk(df, annual_frac = 12):
    tr_df = df.skew().to_frame('Skew')
    tr_df["Kurtosis"] = df.kurtosis()
    tr_df["VaR (.05)"] = df.quantile(.05)
    return tr_df

In [79]:
summary_stats(ltcm_ex[['Gross Performance (Excess)','Net Performance (Excess)','SPY (Excess)']])

,Mean,Vol,Sharpe
Gross Performance (Excess),0.2421,0.1362,1.7769
Net Performance (Excess),0.1554,0.1118,1.3901
SPY (Excess),0.1738,0.1123,1.5479


In [80]:
tail_risk(ltcm_ex[['Gross Performance (Excess)','Net Performance (Excess)','SPY (Excess)']])

,Skew,Kurtosis,VaR (.05)
Gross Performance (Excess),-0.2877,1.5866,-0.0304
Net Performance (Excess),-0.8102,2.9269,-0.0264
SPY (Excess),-0.4335,-0.3620,-0.0464


Question 2

Estimate the following regression:

\begin{align}
\tilde{r}^{LTCM} = \alpha + \beta^{m}\tilde{r}^{m}_{t} +\epsilon_t
\end{align}

In [81]:
def regress(y, x, intercept = True, annual_frac = 12):
    if intercept == True:
        X_ = sm.add_constant(x)
        reg = sm.OLS(y,X_).fit()
        reg_df = reg.params.to_frame("Regression Parameters")
        reg_df.loc['R-squared'] = reg.rsquared
        reg_df.loc['const'] *= annual_frac
    else:
        reg = sm.OLS(y,x).fit()
        reg_df = reg.params.to_frame("Regression Parameters")
        reg_df.loc['R-squared'] = reg.rsquared
        
    return reg_df

In [82]:
regress(ltcm_ex['Net Performance (Excess)'],ltcm_ex['SPY (Excess)'])

,Regression Parameters
const,0.1315
SPY (Excess),0.1371
R-squared,0.0190


Question 3
Quadratic Regression

\begin{align}
\tilde{r}^{LTCM} = \alpha + \beta_{1}\tilde{r}^{m}_{t} + \beta_{2}(\tilde{r}^{m}_{t})^{2} +\epsilon_t
\end{align}

In [83]:
ltcm_ex['SPY (Excess) squared'] = ltcm_ex['SPY (Excess)']**2

regress(ltcm_ex['Net Performance (Excess)'],ltcm_ex[['SPY (Excess)','SPY (Excess) squared']])


,Regression Parameters
const,0.1550
SPY (Excess),0.1669
SPY (Excess) squared,-1.9267
R-squared,0.0243


Question 4

Asymmetric Regression:

\begin{align}
\tilde{r}^{LTCM} = \alpha + \beta\tilde{r}^{m}_{t} + \beta_{u}max(\tilde{r}^{m}_{t} - k_{1},0) + + \beta_{d}max(k_{2} - \tilde{r}^{m}_{t},0) +\epsilon_t
\end{align}

In [84]:
k1 = 0.03
k2 = -.03
ltcm_ex['UP'] = (ltcm_ex['SPY (Excess)'] - k1).clip(0)
ltcm_ex["Down"] = (k2 - ltcm_ex['SPY (Excess)']).clip(0)

In [85]:
regress(ltcm_ex['Net Performance (Excess)'], ltcm_ex[['SPY (Excess)','UP','Down']])

,Regression Parameters
const,0.1012
SPY (Excess),0.4666
UP,-0.7821
Down,1.2896
R-squared,0.0555


# The FX Carry Trade

In [86]:
fx_carry_file = 'C:/Users/dcste/OneDrive/Portfolio_Theory/Homework_Jupyter/portfolio_theory/fx_carry_data.xlsx'
rf = pd.read_excel(fx_carry_file,sheet_name = 1).set_index("DATE")
log_rf = np.log(1+rf)

log_rf.head()

,USD1M,GBP1M,EUR1M,CHF1M,JPY1M
DATE,,,,,
1999-01-31,0.0041,0.0049,0.0026,0.0010,3.3458e-04
1999-02-28,0.0041,0.0046,0.0026,0.0010,2.3226e-04
1999-03-31,0.0041,0.0044,0.0025,0.0010,1.4270e-04
1999-04-30,0.0041,0.0044,0.0021,0.0008,9.8953e-05
1999-05-31,0.0041,0.0044,0.0021,0.0008,7.4997e-05


In [87]:
fx = pd.read_excel(fx_carry_file, sheet_name = 2).set_index("DATE")
log_fx = np.log(fx)

log_fx.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,0.4982,0.1285,-0.3484,-4.7536
1999-02-28,0.4717,0.0949,-0.3712,-4.7766
1999-03-31,0.4787,0.0777,-0.3904,-4.7743
1999-04-30,0.4753,0.0549,-0.4225,-4.7827
1999-05-31,0.4713,0.0413,-0.4240,-4.7948


In [88]:
(log_fx.diff()).head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,NaN,NaN,NaN,NaN
1999-02-28,-0.0265,-0.0336,-0.0228,-0.0230
1999-03-31,0.0070,-0.0172,-0.0191,0.0023
1999-04-30,-0.0034,-0.0228,-0.0321,-0.0084
1999-05-31,-0.0040,-0.0135,-0.0015,-0.0121


## The Static Carry Trade

In [89]:
log_rf_ex = log_rf.subtract(log_rf['USD1M'], axis = 0)[log_rf.columns[1:]]

log_rf_ex_col = log_rf_ex.copy()
log_rf_ex_col.columns = log_fx.columns

ret = (log_rf_ex_col+ log_fx.diff()).dropna()
ret.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-0.0260,-0.0352,-0.0259,-0.0269
1999-03-31,0.0073,-0.0188,-0.0222,-0.0017
1999-04-30,-0.0031,-0.0248,-0.0354,-0.0124
1999-05-31,-0.0037,-0.0155,-0.0048,-0.0161
1999-06-30,-0.0162,-0.0130,-0.0212,-0.0048


In [90]:
summary_stats(ret)

,Mean,Vol,Sharpe
USUK,-0.0035,0.0864,-0.0409
USEU,-0.0043,0.0948,-0.0455
USSZ,0.0044,0.0988,0.0447
USJP,-0.0173,0.0916,-0.1884


## Predicting FX

- For each foreign currency, test whether interest-rate differentials can predict growth in the **FX** rate for currency i. 



In [98]:
yx

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-0.0265,-0.0336,-0.0228,-0.0230
1999-03-31,0.0070,-0.0172,-0.0191,0.0023
1999-04-30,-0.0034,-0.0228,-0.0321,-0.0084
1999-05-31,-0.0040,-0.0135,-0.0015,-0.0121
1999-06-30,-0.0160,-0.0108,-0.0178,-0.0005
...,...,...,...,...
2021-06-30,-0.0273,-0.0288,-0.0283,-0.0110
2021-07-31,0.0077,0.0013,0.0210,0.0122
2021-08-31,-0.0120,-0.0054,-0.0110,-0.0032


In [97]:
yx = log_fx.diff().dropna()
Xs = -log_rf_ex.shift().dropna()

fx_reg_pred = pd.DataFrame(data = None, index = ['Alpha','Beta','R-squared'])

for i in range(0,len(yx.columns)):
    fx_reg_pred[yx.columns[i]] = regress(yx[yx.columns[i]], Xs[Xs.columns[i]])['Regression Parameters'].values
    
fx_reg_pred

,USUK,USEU,USSZ,USJP
Alpha,-0.0059,0.0070,0.0436,-0.0060
Beta,0.4858,-1.2564,-1.6466,0.3715
R-squared,0.0004,0.0026,0.0039,0.0005


## The Dynamic Carry Trade

\begin{align}
\mathbb{E}_{t}[{s}_{t+1} - {s}_{t}] = \alpha + (\beta-1) ({r}^{f,\$}_{t,t+1} -  {r}^{f,i}_{t,t+1})
\end{align}

In [103]:
alphas = fx_reg_pred.loc['Alpha']/12
betas = fx_reg_pred.loc['Beta']

Xs.columns = fx_reg_pred.columns

ret_ex_forecast = alphas.values + ((betas-1)*Xs)

In [104]:
ret_ex_forecast.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-6.8394e-05,-0.0028,-0.0046,-0.0029
1999-03-31,-2.3875e-04,-0.0029,-0.0046,-0.0029
1999-04-30,-3.2543e-04,-0.0031,-0.0046,-0.0030
1999-05-31,-3.2588e-04,-0.0038,-0.0050,-0.0030
1999-06-30,-3.3511e-04,-0.0039,-0.0050,-0.0030


- Use your regression estimates from problem three along with the formula above to calculate the fraction of months for which the estimated **FX** risk premium is positive. 

\begin{align}
\mathbb{E}_{t}[\tilde{r}^{i}_{t+1}] > 0
\end{align}

In [105]:
month_frac = pd.DataFrame(data = None, columns = ret_ex_forecast.columns, index = ['% of Months'])

for col in ret_ex_forecast.columns:
    month_frac[col] = (len(ret_ex_forecast[ret_ex_forecast[col] > 0]))/(len(ret_ex_forecast))*100
    
month_frac

,USUK,USEU,USSZ,USJP
% of Months,23.8095,50.1832,63.0037,0.0
